# **Music Genre Classification**


Copyright @ 2020 **ABCOM Information Systems Pvt. Ltd.** All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and limitations under the License.

# Install **python_speech_features** module.

In [ ]:
!pip install python_speech_features

# pudub for converting mp3 to wav

In [ ]:
!pip install pydub

In [ ]:
import numpy as np
import os
import pickle
import random
import pandas as pd
import sklearn
import scipy.io.wavfile as wav
from os import path
from pydub import AudioSegment
from python_speech_features import mfcc
from sklearn.metrics import classification_report
from collections import defaultdict

# Downloading data

In [ ]:
!wget https://github.com/abcom-mltutorials/music-genre/archive/master.zip

In [ ]:
!unzip "/content/master.zip"

# Creating a list for filenames

In [ ]:
directory = "/content/music-genre-master/Dataset"
filelist=[]
for path, subdirs, files in os.walk(directory):
    for file in files:
        if (file.endswith('.wav') or file.endswith('.WAV')):
            filelist.append(os.path.join(path, file))
number_of_files=len(filelist)
print(number_of_files)

# Extracting features
Extract the feactures from the audio files and store it along with the class_label.

Create a list and append all the features into a single list.

In [ ]:
def feature_extraction(file):
  features=[]
  (sampleRate,data) = wav.read(file)
  mfcc_feature = mfcc(data,sampleRate, 
                            winlen=0.020, 
                            appendEnergy = False)
  meanMatrix = mfcc_feature.mean(0)
  for x in meanMatrix:
    features.append(x)
  return features

In [ ]:
datasetDirectory = "/content/music-genre-master/Dataset/"

featureSet=[]
i=0
for folder in os.listdir(datasetDirectory):
    i+=1
    if i > 9: # the number of genre is 9
        break   
    for files in os.listdir(datasetDirectory+folder):
      x=datasetDirectory+folder+"/"+files
      features=feature_extraction(x)
      j=0
      for x in features:
        featureSet.append(x)
        j=j+1
        if(j%13==0): # the number of feaatures is 13
          featureSet.append(i)

In [ ]:
for i in range(14,28):
  print (featureSet[i])

## Construct dataframe

In [ ]:
df = pd.DataFrame(columns=['m1','m2','m3','m4','m5','m6','m7',
                           'm8','m9','m10','m11','m12','m13','target'])

In [ ]:
i=1
n=[]
for j in featureSet:
  n.append(j)
  #13 features + 1 taget
  if(i%14==0):
    df = df.append({'m1':n[0],'m2':n[1],'m3':n[2],'m4':n[3],'m5':n[4],
                    'm6':n[5],'m7':n[6],'m8':n[7],'m9':n[8],'m10':n[9],
                    'm11':n[10],'m12':n[11],'m13':n[12],'target':n[13]}, 
                   ignore_index=True)
    n=[]
  i=i+1

In [ ]:
df

# Separating features and target

In [ ]:
x1=df[['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12','m13']]
x1.shape

In [ ]:
Y = df[['target']]
Y.shape

# Splitting dataset
Split the data into random train and test subsets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x1, Y, 
                                                    test_size=0.2, 
                                                    random_state=42)

# Use Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train,y_train)

# Predicting

Predict the values for the X_test. The output is array of the class_labels, where 1-9 indicates different genres of the music.

In [ ]:
predicted_value = clf.predict(X_test)
predicted_value

# Confusion Matrix

In [ ]:
sklearn.metrics.plot_confusion_matrix(clf, X_test, y_test)

# Tabulate metrics

In [ ]:
print(classification_report(y_test, predicted_value))

# Unseen audio file

In [ ]:
#Extract the feature from the audio_file
audio_file="/content/music-genre-master/new_audio_file.wav"
audio_feature=feature_extraction(audio_file)

In [ ]:
result={1: 'metal',2: 'classical',3: 'rock', 4: 'disco',5: 'pop',6: 'blues',7: 'reggae',8: 'hiphop',9: 'country'}
pred=clf.predict([audio_feature])
result[int(pred)]

# Your own audio files

In [ ]:
!wget https://raw.githubusercontent.com/abcom-mltutorials/music/master/bhambhole.mp3

In [ ]:
src = "/content/bhambhole.mp3"
dst = "test.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

## Predict the genre of the song.

In [ ]:
#Extract the feature from the audio_file
audio_file="/content/test.wav"
audio_feature=feature_extraction(audio_file)

In [ ]:
pred=clf.predict([audio_feature])
result[int(pred)]